# Tutorial: Use trained model to perform recommendation.

Sample JSON Requests:
```json
json_request = {
'专业': ['Angular', 'React'],
'专业证书': [],
'学历': [],
'所在领域': ['前端开发'],
'技术栈': ['Angular', 'React'],
'数量': 1,
'职位': '前端开发工程师',
'职位工作年限': [],
'职称名称': [],
'语言要求': [],
'项目经验': []
}
```

In [1]:
json_request = {
    '专业': ['Angular', 'React'],
    '专业证书': [],
    '学历': [],
    '所在领域': ['前端开发'],
    '技术栈': ['Angular', 'React'],
    '数量': 1,
    '职位': '前端开发工程师',
    '职位工作年限': [],
    '职称名称': [],
    '语言要求': [],
    '项目经验': []
}

We use predefined configurations which is stored at `./configs.yaml`
```python
# use the following codes to get a view of our configuration.
configs = "./talent_recommendation/configs.yaml"
configs = load_configs(configs, check_keys=["neo4j"])
model, data = rec_sys_train(configs)
```

## One Step Function Calling: If you already has your model and data

`similarity_compute()` is a one-stop function which could reload pre-trained recommendation system GNN model and HeteroData object according to our configuration file.

In [2]:
from talent_recommendation.BipartiteRecSys import similarity_compute

df, out = similarity_compute(json_request, configs="./configs.yaml")
df

No sentence-transformers model found with name /root/KnowledgeAlgorithms/models/sbert-base-chinese-nli. Creating a new one with MEAN pooling.


Found HeteroData at /root/KnowledgeAlgorithms/checkpoints/dataset.pt
2 new edges added.
Employee ID: 2962, Name: MT11551, Score: 4.133079528808594
Employee ID: 7018, Name: MT51801, Score: 3.986539602279663
Employee ID: 2833, Name: MT44283, Score: 3.5716400146484375
Employee ID: 9065, Name: MT34635, Score: 3.5663764476776123
Employee ID: 4523, Name: MT43368, Score: 3.525930166244507


/root/KnowledgeAlgorithms/talent_recommendation/BipartiteRecSys.py:764: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3614.)
  scores = torch.matmul(employee_embeddings, position_embedding.T)


,工号,学历,首次工作时间,年龄
0,MT11551,本科,None,None
1,MT51801,本科,None,None
2,MT44283,大专,None,None
3,MT34635,大专,None,None
4,MT43368,本科,None,None


## Train from beginning.

If you don't have your model and data at hand, but at least have your neo4j usr name and password. You could train from begginning. There are functions calls that could do the whole training process step by step. Please refer to the documentation. For convenience, we still use `similarity_compute()` functions to do the whole process.

In [3]:
configs = {'dataTransform': {'edge_types': [['employee', 'workedAs', 'position']],
  'is_undirected': True,
  'neg_sampling_ratio': 0.0,
  'num_test': 0.0,
  'num_val': 0.1,
  'random_data_split': True,
  'rev_edge_types': [['position', 'rev_workedAs', 'employee']]},
 'graphNeuralNetwork': {'hidden_channels': 128,
  'input_channels': 768,
  'out_channels': 64},
 'neo4j': {'labels': ['employee', 'project', 'position', 'techStack'],
  'password': 'tanzhezhineng',
  'url': 'bolt://localhost:7687',
  'user': 'neo4j'},
 'save': None,
 'sentence_encoder': '/root/KnowledgeAlgorithms/models/sbert-base-chinese-nli',
 'targetEdgeType': '("position", "rev_workedAs", "employee")',
 'train': {'k_to_hit': 10, 'num_epochs': 50, 'optimizer': 'Adam'}}

In [4]:
similarity_compute(json_request, configs)

No dataset found. Fetch data from neo4j database.
Loading node:employee ...	 10000 nodes loaded.
Loading node:project ...	  4371 nodes loaded.
Loading node:position ...	    41 nodes loaded.
Loading node:techStack ...	    66 nodes loaded.
Loading relation (employee)-->(project) ...	 39986 loaded.
Loading relation (employee)-->(position) ...	 38513 loaded.
Loading relation (employee)-->(techStack) ...	  2966 loaded.
Loading relation (project)-->(employee) ...	     0 loaded.
Loading relation (project)-->(position) ...	 35641 loaded.
Loading relation (project)-->(techStack) ...	 37316 loaded.
Loading relation (position)-->(employee) ...	     0 loaded.
Loading relation (position)-->(project) ...	     0 loaded.
Loading relation (position)-->(techStack) ...	  2665 loaded.
Loading relation (techStack)-->(employee) ...	     0 loaded.
Loading relation (techStack)-->(project) ...	     0 loaded.
Loading relation (techStack)-->(position) ...	     0 loaded.
No model could be found. Now we are going 

/root/miniconda3/envs/recsys/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch [10/50], Loss: 1.248204, Val Loss: 1.6661 Val AUC: 0.5095, Val Precision: 0.0000, Val Recall: 0.0000, Val F1: 0.0000, Val Hit@10: 0.0000
Epoch [20/50], Loss: 3.043222, Val Loss: 1.9877 Val AUC: 0.5570, Val Precision: 0.5018, Val Recall: 0.2961, Val F1: 0.3724, Val Hit@10: 0.8000
Epoch [30/50], Loss: 0.419615, Val Loss: 0.4837 Val AUC: 0.6384, Val Precision: 0.8059, Val Recall: 0.0228, Val F1: 0.0443, Val Hit@10: 0.9000
Epoch [40/50], Loss: 0.336543, Val Loss: 0.3640 Val AUC: 0.6999, Val Precision: 0.6490, Val Recall: 0.6214, Val F1: 0.6349, Val Hit@10: 0.9000


No sentence-transformers model found with name /root/KnowledgeAlgorithms/models/sbert-base-chinese-nli. Creating a new one with MEAN pooling.


Epoch [50/50], Loss: 0.295100, Val Loss: 0.2966 Val AUC: 0.7458, Val Precision: 0.8526, Val Recall: 0.0986, Val F1: 0.1768, Val Hit@10: 1.0000
2 new edges added.
Employee ID: 23, Name: MT47122, Score: 2.9764935970306396
Employee ID: 154, Name: MT53926, Score: 2.6382429599761963
Employee ID: 58, Name: MT37925, Score: 2.5436573028564453
Employee ID: 310, Name: MT18723, Score: 2.5149214267730713
Employee ID: 283, Name: MT86493, Score: 2.4734463691711426


(        工号  学历 首次工作时间    年龄          技术栈
 0  MT47122  大专   None  None      [Jetty]
 1  MT53926  本科   None  None        [ant]
 2  MT37925  大专   None  None  [FreeMaker]
 3  MT18723  本科   None  None       [jdbc]
 4  MT86493  硕士   None  None  [SecureCRT],
 [('MT47122', 2.9764935970306396),
  ('MT53926', 2.6382429599761963),
  ('MT37925', 2.5436573028564453),
  ('MT18723', 2.5149214267730713),
  ('MT86493', 2.4734463691711426)])